In [1]:
import itertools
import numpy as np
from gurobipy import Model, GRB

def solve_linear_program():
    model = Model("Linear_Programming_Example")
    
    attributes = ["A", "B", "C", "D", "E", "F", "G", "H"]
    relations = [["A","B","C","D"], ["A","B","E","F"], ["C","D","E","F"], ["A","B","G","H"], ["C","D","G","H"]]
    
    #attributes = ["x", "y", "z"]
    #relations = [["x", "y"], ["y", "z"], ["z", "x"]]
    
    #attributes = ["A", "B", "X", "Y", "C"]
    #relations = [["A","B","X","Y", "C"], ["X","Y"], ["A","X"], ["A","Y"], ["B","X"], ["B", "Y"], ["C"]]
    
    #attributes = ["A", "B", "C", "D", "E"]
    #relations = [["A", "B"], ["B", "C"], ["C", "D"], ["D", "E"]]
    
    #attributes = ["x", "y", "z"]
    #relations = [["x", "y"], ["y", "z"]]
    
    #attributes = ["x", "y", "z", "u"]
    #relations = [["x", "y"], ["y", "z"], ["z", "u"]]
    
    variables = {}
    for i in range(1, len(attributes) + 1):
        for rho_set in itertools.combinations(attributes, i):
            if frozenset(rho_set) not in variables:
                rho = "".join(rho_set)
                name = f'D({rho}||{"".join(attributes)})'
                x = model.addVar(lb=0, name=name)
                #name2 = f"log({rho})"
                #y = model.addVar(lb=0, ub=len(rho_set), name=name2)
                variables[frozenset(rho_set)] = x
    
    #objective = variables[frozenset(attributes)][0] - variables[frozenset(attributes)]
    #model.addConstr(objective <= len(attributes), "objective_bounded")
    
    sum_of_relations = 0
    sum_of_relative_entropies = 0
    for relation in relations:
        #model.addConstr(variables[frozenset(relation)] >= 1, f"rel_{"".join(relation)}")
        #model.addConstr(variables[frozenset(relation)][0] >= 1, f"rel_lower_{"".join(relation)}")
        #model.addConstr(variables[frozenset(relation)][1] >= 1, f"rel__entropy_upper_{"".join(relation)}")
        sum_of_relations += variables[frozenset(relation)]
        #sum_of_relative_entropies += variables[frozenset(relation)][1]
        #model.addConstr(variables[frozenset(attributes)][0] <= len(relations)*variables[frozenset(relation)][0], f"upper_bound_{"".join(relation)}")
        
    model.setObjective(sum_of_relations, GRB.MINIMIZE)
    
    for a in attributes:
        relations_with_a = [relation for relation in relations if a in relation]
        sum_of_relations_with_a = 0
        for relation in relations_with_a:
            sum_of_relations_with_a += variables[frozenset(relation)]
        vars_in_relations = []
        for relation in relations_with_a:
            vars_in_relations += relation
        vars_in_relations = set(vars_in_relations)
        model.addConstr(sum_of_relations_with_a >= 1, f"sum_of_relations_{a}")
    
    #for a in attributes:
    #    model.addConstr(variables[frozenset([a])] >= 1, f"single_attribute_{a}")
        
    #for a in attributes:    
    #    relations_with_a = [relation for relation in relations if a in relation]
    #    sum_of_relations_with_a = 0
    #    sum_of_relative_entropies_with_a = 0
    #    for relation in relations_with_a:
    #        sum_of_relations_with_a += variables[frozenset(relation)][0]
    #        sum_of_relative_entropies_with_a += variables[frozenset(relation)][1]
    #    model.addConstr(sum_of_relations_with_a >= 1, f"sum_of_relations_{a}")
    #    model.addConstr(sum_of_relative_entropies_with_a >= 1, f"sum_of_relative_entropies_{a}")
    
    #model.addConstr(variables[frozenset(attributes)][0] <= sum_of_relations, "sum_of_relations")
    #model.addConstr(variables[frozenset(attributes)][1] <= sum_of_relative_entropies, "sum_of_relative_entropies")
    
        
    #fds = [[["E","F","G","H"], ["A","B","C","D"]], [["A","B","C","E"], ["D","F","G","H"]], [["B","C","D","G"], ["A","E","F","H"]]]
    #candidate_keys = [[["E","F","G","H"], ["A","B","C","D", "E","F","G","H"]], [["A","B","C","E"], ["D","F","G","H", "A","B","C","E"]], [["B","C","D","G"], ["A","E","F","H", "B","C","D","G"]]]
    #fds = fds + candidate_keys
    #for Y, x in fds:
    #    Yx = Y + x
    #    model.addConstr(variables[(frozenset(Y), frozenset(Yx))] == 0, "fd_" + "".join(Y) + "->" + "".join(x))
    
    # Monotonicty of relative entropy
    for rho in variables:
        if len(rho) > 1:
            for traced_out_element in rho:
                rho_prime = rho - frozenset([traced_out_element])
                model.addConstr(variables[rho] >= variables[rho_prime], f"monotonicity_{"".join(rho_prime)}_{traced_out_element}")
                
    # Monotonicity of log
    #if True:
    #    for rho in variables:
    #        if len(rho) > 1:
    #            for traced_out_element in rho:
    #               rho_prime = rho - frozenset([traced_out_element])
    #                model.addConstr(variables[rho][0] >= variables[rho_prime][0] + variables[frozenset(traced_out_element)][0], f"log_monotonicity_{"".join(rho_set)}_{"".join(rho_prime)}")
    
    
    model.write("relative_entropy_program1.lp")
    model.optimize()

    # Check optimization status
    if model.status == GRB.OPTIMAL:
        print("Optimal solution found!")
        print(f"Objective value = {model.objVal}")
        print(f"Variables:")
        for var in model.getVars():
            if var.x != 0:
                print(f"{var.varName} = {var.x}")
        return model.objVal
    elif model.status == GRB.INFEASIBLE:
        print("The model is infeasible.")
    elif model.status == GRB.UNBOUNDED:
        print("The model is unbounded.")
    else:
        print(f"Optimization ended with status {model.status}")
    
result = solve_linear_program()
print(f"Result: {result}")

Set parameter Username
Academic license - for non-commercial use only - expires 2025-11-26
Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (win64 - Windows 10.0 (19045.2))
Academic license - for non-commercial use only - expires 2025-11-26
Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (win64 - Windows 10.0 (19045.2))

CPU model: AMD Ryzen 7 5800H with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 1024 rows, 255 columns and 2052 nonzeros
Model fingerprint: 0xa74dd342
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 1020 rows and 250 columns
Presolve time: 0.00s
Presolved: 4 rows, 5 columns, 10 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   4.000000e+00   0.000000e+00      0s
       4    2.0000000e+00 